# Imports

In [1]:
from pathlib import Path
from typing import Literal

import numpy as np
import torch
from tqdm.auto import tqdm

from Datasets.base import DatasetMode
from Datasets.batching import BatchManager
from mean_average_precision import MetricBuilder

C:\Users\benia\anaconda3\envs\GNNs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parameter Setup

In [2]:
# Choose an investigated dataset and model
dataset_name: Literal["NCaltech101", "Gen1"] = "Gen1"
model_name: Literal["EGSST"] = "EGSST"

# Set corresponding paths
dataset_path: Path = Path(r"D:\Uniwersytet\GNNBenchmarking\Datasets\GEN1-DS")
model_path: Path = Path(r"C:\Users\benia\Desktop\Gen1_egsst_trained_epoch_499.pth")

# mAP evaluation parameters
batch_size: int = 8
investigated_mode: DatasetMode = "validation"
epoch_count: int = 10

# Graph preprocessing parameters
inference_event_count: int = 10000
beta: float = 0.0001
radius: float = 5.

# EGSST specific parameters
min_nodes_subgraph: int = 1000
detection_head_config: Path = Path(r"../confs/rtdtr_head_gen1.yml")
ecnn_flag: bool = False # Whether enhanced cnn should be used
ti_flag: bool = False # Whether TAC augmentation should be used

# Loading Dataset

In [3]:
print(f"Loading {dataset_name} dataset...")
if dataset_name == "Gen1":
    from Datasets.gen1 import Gen1
    dataset = Gen1(
        root=dataset_path,
    )
elif dataset_name == "NCaltech101":
    from Datasets.ncaltech101 import NCaltech
    dataset = NCaltech(
        root=dataset_path,
    )
else:
    raise ValueError(f"Dataset {dataset_name} not implemented.")

dataset.process()
print(f"Dataset Initialized.")

batch_manager = BatchManager(
    dataset = dataset,
    batch_size = batch_size,
    mode = investigated_mode
)

Loading Gen1 dataset...


validation (val_b): 100%|██████████| 179/179 [00:00<00:00, 5755.58it/s]

Dataset Initialized.


# Initializing Model

In [4]:
print(f"Initializing {model_name} model...")
if model_name == "EGSST":
    from Models.EGSST.EGSST import EGSST
    model = EGSST(
        dataset_information = dataset.get_info(),
        detection_head_config = str(detection_head_config)
    )

    def transform_graph(graph):
        graph.x = graph.x[:inference_event_count, :]
        graph.pos = graph.pos[:inference_event_count, :]
        graph = model.data_transform(graph, beta = beta, radius = radius, min_nodes_subgraph = min_nodes_subgraph)

        if graph is None or graph.pos is None:
            return None

        if isinstance(graph.bbox, dict):
            maximum_time = graph.pos[:, 2].max()
            times_of_boxes = torch.tensor(list(graph.bbox.keys()))
            time_diff = (times_of_boxes - maximum_time) ** 2
            graph.bbox = graph.bbox[list(graph.bbox.keys())[time_diff.argmin()]]

        return graph

    graph_transform = transform_graph
else:
    raise ValueError(f"Model {model_name} not implemented.")
print(f"Model Initialized.")

print(f"Loading model from: {model_path}")
state_dict = torch.load(model_path, map_location=torch.device("cpu"))
model = model.load_state_dict(state_dict)
print("Model Loaded.")

dataset.transform = graph_transform

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {device}")

Initializing EGSST model...


C:\Users\benia\PycharmProjects\GNNBenchmark\src\External\EGSST_PAPER\detector\efvit\nn\ops.py:570: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


Model Initialized.
   Using device: cuda


# mAP Computation

In [5]:
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=False, num_classes=len(dataset.get_info().classes))

model.eval()
with torch.no_grad():
    for epoch in tqdm(range(1, epoch_count + 1), total = epoch_count, desc = f"Assessing mAP"):
        batch = next(batch_manager)
        batch = batch.to(device)
        output = model(batch)

        for i in range(batch.num_graphs):
            graph_data = batch.get_example(i)
            bbox = graph_data.bbox.detach().cpu().numpy()
            gt_label = bbox[:, 0]
            gt_x_min, gt_x_max = bbox[:, 1] - bbox[:, 3]/2, bbox[:, 1] + bbox[:, 3]/2
            gt_y_min, gt_y_max = bbox[:, 2] - bbox[:, 4]/2, bbox[:, 2] + bbox[:, 4]/2
            gt_bbox = np.concat(
                [
                    gt_x_min[:, None],
                    gt_y_min[:, None],
                    gt_x_max[:, None],
                    gt_y_max[:, None],
                    gt_label[:, None],
                    np.zeros_like(gt_label[:, None]),
                    np.zeros_like(gt_label[:, None]),
                ],
                axis = 1
            )

            pred_boxes = output[i]
            pred_boxes = torch.concat(
                [
                    pred_boxes["boxes"],
                    pred_boxes["labels"][:, None],
                    pred_boxes["scores"][:, None],
                ],
                dim = 1
            ).detach().cpu().numpy()

            metric_fn.add(pred_boxes, gt_bbox)

print(f"MAP@0.9: {metric_fn.value(iou_thresholds = 0.9)['mAP']}")
print(f"MAP@0.5: {metric_fn.value(iou_thresholds = 0.5)['mAP']}")
print(f"MAP@0.25: {metric_fn.value(iou_thresholds = 0.25)['mAP']}")
print(f"MAP@0.1: {metric_fn.value(iou_thresholds = 0.1)['mAP']}")
print(f"MAP@0.05: {metric_fn.value(iou_thresholds = 0.05)['mAP']}")

Assessing mAP:   0%|          | 0/10 [00:00<?, ?it/s]C:\Users\benia\anaconda3\envs\GNNs\Lib\site-packages\mean_average_precision\mean_average_precision_2d.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.match_table[c] = pd.concat([self.match_table[c], match_table], axis=0, join='outer')
Assessing mAP: 100%|██████████| 10/10 [01:26<00:00,  8.61s/it]

MAP@0.9: 0.0
MAP@0.5: 2.648866211529821e-05
MAP@0.25: 0.013835129328072071
MAP@0.1: 0.0849798321723938
MAP@0.05: 0.17857615649700165
